In [3]:
import requests
year = 2022
day = 16

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open('cookie.txt', 'r') as file:
    session = file.read()

headers = {'cookie': f"session={session}"}

with open(f'day_{day}_input.txt', 'wb') as file:
    file.write(requests.get(url, headers=headers).content)
    
import os
if os.path.exists(f"day_{day}_test.txt"):
    test = [x.strip() for x in open(f'day_{day}_test.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
if os.path.exists(f"day_{day}_test_2.txt"):
    test_2 = [x.strip() for x in open(f'day_{day}_test_2.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
lines = [x.strip() for x in open(f'day_{day}_input.txt', 'r')]
for line in lines[:10]:
    print(line.strip())

---
---
Valve SZ has flow rate=0; tunnels lead to valves GQ, YZ
Valve SP has flow rate=0; tunnels lead to valves LJ, AA
Valve LQ has flow rate=0; tunnels lead to valves EY, JT
Valve AT has flow rate=17; tunnels lead to valves DX, BU, NE, BR, TD
Valve IR has flow rate=0; tunnels lead to valves XN, UI
Valve CF has flow rate=0; tunnels lead to valves XN, BR
Valve TE has flow rate=0; tunnels lead to valves YA, RY
Valve GQ has flow rate=22; tunnels lead to valves SZ, AQ, OW, XJ
Valve DX has flow rate=0; tunnels lead to valves HI, AT
Valve AQ has flow rate=0; tunnels lead to valves AZ, GQ


In [4]:
import re

run_test = False

if run_test:
    case = test
else:
    case = lines
    
V = {}
L = []
for line in case:
    pattern = r"Valve (\S+) has flow rate=(\d+); tunnels? leads? to valves? ([\S, ]+)"
    groups = re.findall(pattern,line)
    s = groups[0][0]
    p = int(groups[0][1])
    ts = [x.strip() for x in groups[0][2].split(',')]
    V[s] = p
    for t in ts:
        L.append((s,t))

In [5]:
from pprint import pprint
#print(V)

# At each step, options are:
    # move, open
    
DP = dict()

num_lookups = 0
num_calcs = 0

def advance_time(state, current_pressure_released):
    global num_calcs, num_lookups
    position = state[0]
    time = state[1]
    current_open_valves = state[2]
    
    if time == 30:
        #assert False
        #print(current_pressure_released)
        return current_pressure_released
    
    if state in DP:
        num_lookups+=1
        return DP[state]
    else:
        num_calcs+=1
        current_pressure_released += sum([V[v] for v in current_open_valves])
        #print(state, current_pressure_released)

        # Move
        for l in [l[1] for l in L if l[0] == position]:
            n_state = (l, time+1, current_open_valves)
            advance_time(n_state, current_pressure_released)
            #DP[n_state] = current_pressure_released

        # Open
        for l in [l[0] for l in L if l[0] == position and l[0] not in current_open_valves and V[l[0]] > 0]:
            n_state = (position, time+1, current_open_valves + tuple([l]))
            advance_time(n_state, current_pressure_released)
        
        DP[state] = current_pressure_released
        #return DP[state]

advance_time(('AA',0,()),0)
# Too slow
#pprint(DP)
print(num_lookups,num_calcs)
print(max(DP.values()))


5575901 5576537
1828


In [31]:
from itertools import combinations
t = sorted({(k[2],v) for k,v in DP.items() if k[1] == 25}, key=lambda x:-x[1])#[:1000]

keys = (next(((a,b,sa,sb) for (a, sa), (b, sb) in combinations(t, 2) if not set(a) & set(b))))

print(keys)
print(keys[2]+keys[3])


(('VK', 'GQ', 'YL', 'YA', 'AZ', 'EY', 'JT'), ('UI', 'AT', 'ES', 'TR', 'XN'), 1376, 916)
2292


In [ ]:
from pprint import pprint
from collections import deque
from itertools import chain
    

def advance_time(first_state):
    
    # states are sorted(positions), 
    # time, 
    # current_open_valves
    # current_pressure_released
    
    best_pressure_released = 0
    
    Q = deque([first_state])
    
    states = set()
    
    while Q:
        state = Q.popleft()
        
        #print(state)
        if state in states:
            continue
        states.add(state)
        
        positions = list(state[0])
        time = state[1]
        current_open_valves = state[2]
        current_pressure_released = state[3]
        
        if time == MAX_DURATION:
            continue
        
        if len(states) % 100000 == 0:
            print(time,best_pressure_released,len(states))

        #print(current_open_valves)
        current_pressure_released = current_pressure_released + sum([V[v] for v in current_open_valves])
        
        if current_pressure_released > best_pressure_released:
            best_pressure_released = current_pressure_released
            
        #all valves open - sprint to end.
#         if len(current_open_valves) == len([v for v in V if V[v] > 0]):
#             for i, t in enumerate(range(time+1,MAX_DURATION+1)):
#                 n_state = (positions, t, current_open_valves)
#                 if n_state not in DP:
#                     DP[n_state] = current_pressure_released + i * sum([V[v] for v in current_open_valves])
#             return DP[state]

        searches = list()

        # both open
        if V[positions[0]] > 0 and V[positions[1]] > 0:
            n_state = (tuple(positions), time+1, frozenset(list(current_open_valves) + positions), current_pressure_released)
            searches.append(n_state)

        #you move, elephant opens
        for i in [l[1] for l in L if l[0] == positions[0]]:
            for j in [l[0] for l in L if l[0] == positions[1] and V[positions[1]] > 0]:
                n_state = (tuple(sorted((i,positions[1]))), time+1, frozenset(list(current_open_valves) + [j]), current_pressure_released)
                searches.append(n_state)

        # elephant moves, you open
        for i in [l[0] for l in L if l[0] == positions[0] and V[positions[0]] > 0]:
            for j in [l[1] for l in L if l[0] == positions[1]]:
                n_state = (tuple(sorted((positions[0],j))), time+1, frozenset(list(current_open_valves) + [i]), current_pressure_released)
                searches.append(n_state)


        # both move
        for i in [l[1] for l in L if l[0] == positions[0]]:
            for j in [l[1] for l in L if l[0] == positions[1]]:
                n_state = (tuple(sorted(((i,j)))), time+1, frozenset(current_open_valves), current_pressure_released)
                searches.append(n_state)

        # both stay
#         n_state = (tuple(positions), time+1, frozenset(current_open_valves), current_pressure_released)
#         searches.append(n_state)
        
        #pprint(searches)
        if current_pressure_released > best_pressure_released:
            Q.extendleft(searches)
        else:
            Q.extend(searches)
    
    print(len(states))
    return best_pressure_released
        
MAX_DURATION = 26

solution = advance_time((('AA','AA'), 0, (), 0))

print(solution)

# for i in range(1,26+1):
#     max_day = 0
#     max_key = None
#     for k,v in DP.items():
#         if k[1] == i and v >= max_day:
#             max_day = v
#             max_key = k
#     print(f"{i}: {max_day}, {max_key}")

# 2298 too high
# 2113 too low
# 2292

8 191 100000
9 227 200000
9 239 300000
9 255 400000
10 308 500000
10 308 600000
10 308 700000
10 333 800000
10 333 900000
10 333 1000000
10 342 1100000
10 342 1200000
10 342 1300000
11 342 1400000
11 342 1500000
11 369 1600000
11 389 1700000
11 389 1800000
11 389 1900000
11 389 2000000
11 389 2100000
11 389 2200000
11 397 2300000
11 427 2400000
11 427 2500000
11 427 2600000
11 427 2700000
11 427 2800000
11 427 2900000
11 427 3000000
11 427 3100000
11 427 3200000
11 429 3300000
11 429 3400000
11 429 3500000
11 429 3600000
11 429 3700000
11 429 3800000
11 429 3900000
11 429 4000000
11 429 4100000
11 429 4200000
12 429 4300000
12 429 4400000
12 429 4500000
12 429 4600000
12 429 4700000
12 429 4800000
12 429 4900000
12 450 5000000
12 470 5100000
12 470 5200000
12 470 5300000
12 470 5400000
12 470 5500000
12 470 5600000
12 470 5700000
12 470 5800000
12 470 5900000
12 470 6000000
12 470 6100000
12 470 6200000
12 470 6300000
12 470 6400000
12 470 6500000
12 470 6600000
12 470 6700000
12 470 6